# Rush Hour

## Un modèle pour RushHour

### Un exemple d'utilisation programmatique

In [1]:
from RushHour import *

In [2]:
plateau = Plateau(['A2R00','X2R21','C2R44','R3R52','O3D05','P3D10','Q3D13','B2D40']); plateau

+------+
|AA   O|
|P  Q O|
|PXXQ O 
|P  Q  |
|B   CC|
|B RRR |
+------+

In [3]:
plateau.recule("Q")

+------+
|AA Q O|
|P  Q O|
|PXXQ O 
|P     |
|B   CC|
|B RRR |
+------+

In [4]:
plateau.est_gagnant()

False

Chargement du 40ème défi de RushHour (voir les fichiers dans <a href="RushHourDefis/">RushHourDefis/</a>):

In [5]:
Plateau(40)

+------+
|OAA B |
|OCD BP|
|OCDXXP 
|QQQE P|
|  FEKK|
|HHFII |
+------+

### Un exemple d'interface utilisateur minimale

In [5]:
from ipywidgets import interact_manual
from RushHour import Plateau
plateau = Plateau(['A2R00','X2R21','C2R44','R3R52','O3D05','P3D10','Q3D13','B2D40']); plateau
@interact_manual
def step(voiture=plateau.voitures.keys(), distance=[0,-1,1]):
    global plateau
    plateau2 = plateau.avance(voiture, distance)
    if plateau2:
        plateau = plateau2
    return plateau

interactive(children=(Dropdown(description='voiture', options=('A', 'X', 'C', 'R', 'O', 'P', 'Q', 'B'), value='A'), Dropdown(description='distance', options=(0, -1, 1), value=0), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

## Une application graphique basée sur les widgets de Jupyter

In [6]:
from ipywidgets import Button, Dropdown, VBox, HBox, Textarea
from bqplot import Figure, LinearScale, Lines, Tooltip
import time

In [7]:
class Application:
    def __init__(self):
        self.niveau  = Dropdown(options=[('Niveau {}'.format(i), i) for i in RushHour.niveaux()])
        self.niveau.observe(lambda widget: self.change_niveau(widget.owner.value))
        
        self.voiture = Dropdown(options=[])
        self.modele = Plateau(1)
        self.vue = Figure(scale_x = LinearScale(min=0, max=self.modele.dimension),
                          scale_y = LinearScale(min=self.modele.dimension, max=0))
        self.vue.layout.width="75ex"
        self.vue.layout.width="75ex"
        self.vue.layout.height=self.vue.layout.width;

        self.vue.vue_voitures = {}
        for lettre, couleur in couleurs.items():
            vue_voiture = Lines(x=[], y=[],
                                scales={'x':self.vue.scale_x,
                                        'y':self.vue.scale_y},
                                fill='inside',
                                colors=[couleurs[lettre]],
                                visible=False,
                                tooltip=Tooltip(fields=["lettre"],show_labels=False),
                                )
            vue_voiture.lettre = lettre
            vue_voiture.on_element_click(lambda vue_voiture, _: self.choix_voiture(vue_voiture.lettre))
            self.vue.vue_voitures[lettre] = vue_voiture
        self.vue.marks = list(self.vue.vue_voitures.values())

        boutton_solution = Button(description="Solution")
        boutton_solution.on_click(self.montre_solution)
        self.widget = VBox([HBox([self.niveau, boutton_solution]),
                            self.vue,
                            self.boutton_direction('U'),
                            HBox([self.boutton_direction('L'), self.voiture, self.boutton_direction('R')]),
                            self.boutton_direction('D')
                           ])
        self.widget.layout.align_items = 'center'
        self.change_niveau(1)

    # Vues
    def boutton_direction(self, direction):
        boutton = Button(description=direction)
        boutton.on_click(self.on_click_direction)
        return boutton

    # Contrôleurs
    def change_niveau(self, i):
        self.niveau = i
        self.modele = Plateau(i)
        self.voiture.options = self.modele.voitures.keys()
        for letter, vue_voiture in self.vue.vue_voitures.items():
            vue_voiture.visible = letter in self.modele.voitures
        self.mise_a_jour_vue()
        
    def montre_solution(self, boutton):
        self.modele = Plateau(self.niveau)
        self.mise_a_jour_vue()
        boutton.description="Calcul en cours ..."
        solution = RushHour.solution(self.niveau)
        boutton.description="Solution"
        for coup in solution:
            self.modele = self.modele.deplace(coup)
            self.mise_a_jour_vue()
            time.sleep(1)
        
    def on_click_direction(self, boutton):
        plateau = self.modele.deplace(self.voiture.value, boutton.description)
        if plateau is not None:
            self.modele = plateau
            self.mise_a_jour_vue()
        
    def choix_voiture(self, lettre):
        self.voiture.value = lettre
        
    def mise_a_jour_vue(self):
        for lettre,voiture in self.modele.voitures.items():
            cases = voiture.cases()
            xmin = min(case[1] for case in cases)
            xmax = max(case[1] for case in cases) + 1
            ymin = min(case[0] for case in cases)
            ymax = max(case[0] for case in cases) + 1
            vue_voiture = self.vue.vue_voitures[lettre]
            vue_voiture.x = [xmin, xmax, xmax, xmin]
            vue_voiture.y = [ymin, ymin, ymax, ymax],

    
    def display(self):
        return self.widget

A = Application()
A.display()

VBox(children=(HBox(children=(Dropdown(options=(('Niveau 1', 1), ('Niveau 3', 3), ('Niveau 7', 7), ('Niveau 8', 8), ('Niveau 11', 11), ('Niveau 15', 15), ('Niveau 21', 21), ('Niveau 38', 38), ('Niveau 40', 40)), value=1), Button(description='Solution', style=ButtonStyle()))), Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='75ex', min_width='125px', width='75ex'), marks=[Lines(colors=['red'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, x=array([1, 3, 3, 1]), y=array([2, 2, 3, 3])), Lines(colors=['lightgreen'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, x=array([0, 2, 2, 0]), y=array([0, 0, 1, 1])), Lines(colors=['orange'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, x=array([0, 1, 1, 0]), y=array([4, 4, 6, 6])), Lines(colors=['lightblue'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, x=array([4, 6, 6, 4]), y=array([4, 4, 5, 5])), Lines(colors=['pink'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, visible=False), Lines(colors=['violet'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, visible=False), Lines(colors=['green'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, visible=False), Lines(colors=['gray'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearScale(max=0.0, min=6.0)}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip=Tooltip(fields=['lettre'], show_labels=False), tooltip_style={'opacity': 0.9}, visible=False), Lines(colors=['khaki'], fill='inside', interactions={'hover': 'tooltip'}, scales={'x': LinearScale(max=6.0, min=0.0), 'y': LinearSca

## Exercice

Déterminer une séquence minimale de coup permettant de résoudre le défi 40 de Rush Hour:

In [9]:
### BEGIN SOLUTION
solution = RushHour.solution(40)
### END SOLUTION

In [10]:
RushHour.est_solution(40, solution)

True